In [22]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd

def rsi_func(price_series, n=14):
    series = pd.Series(price_series)
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).ewm(alpha=1/n, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(alpha=1/n, adjust=False).mean()
    rs = gain / (loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def bollinger_bands_indicator(data_series, n=20, nsig=2):
    series = pd.Series(data_series)
    middle_band = series.rolling(window=n).mean()
    std_dev = series.rolling(window=n).std()
    upper_band = middle_band + (std_dev * nsig)
    lower_band = middle_band - (std_dev * nsig)
    return upper_band, middle_band, lower_band

In [23]:
# rsi 이용해서 backtesting

from backtesting import Backtest, Strategy

class RSIANDBB(Strategy):
    rsi_period = 14
    rsi_buy = 30
    bb_period = 20
    bb_std = 2

    def init(self):      # 초기값 지정
        self.rsi = self.I(rsi_func, self.data.Close, self.rsi_period)
        self.upper_band, self.middle_band, self.lower_band = self.I(
            bollinger_bands_indicator, self.data.Close, self.bb_period, self.bb_std
        )

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1] and self.rsi[-1] < self.rsi_buy:      # 현재 가격이 볼린저 밴드 하단보다 아래에 있고 30보다 낮으면 
            if not self.position:
                self.buy()
    
            elif crossover(self.data.Close, self.middle_band):
                if self.position:
                    self.position.close()

In [24]:
import FinanceDataReader as fdr
df = fdr.DataReader("AAPL", "2020")

bt = Backtest(df, RSIANDBB, cash = 100000, commission=0.002)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/1361 [00:00<?, ?bar/s]

Start                     2019-12-31 00:00:00
End                       2025-06-30 00:00:00
Duration                   2008 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                 318266.59324
Equity Peak [$]                  401787.92904
Return [%]                          218.26659
Buy & Hold Return [%]               153.03077
Return (Ann.) [%]                    23.52292
Volatility (Ann.) [%]                40.77832
CAGR [%]                             15.63765
Sharpe Ratio                          0.57685
Sortino Ratio                         1.09039
Calmar Ratio                          0.70365
Alpha [%]                            70.14151
Beta                                  0.96794
Max. Drawdown [%]                   -33.42972
Avg. Drawdown [%]                    -5.68966
Max. Drawdown Duration      525 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                    0
Win Rate [%]                      

# RSI 활용한 머신러닝 모델
50일치(시가, 고가, 종가, 저가, 거래량, 변화율) 썼을 때.......... 정밀도 약 30%, 재현율 1~2%

In [25]:
# ETF: 여러개의 종목을 묶어놓은 것. VOO: 미국1등~500등

etfs = fdr.StockListing("ETF/KR")
etfs

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
0,459580,6,KODEX CD금리액티브(합성),1073465,2,65,0.01,1073460.0,0.6809,153997,165310,84858
1,360750,4,TIGER 미국S&P500,20875,2,50,0.24,20897.0,0.4292,5553894,115872,82717
2,069500,1,KODEX 200,42155,2,455,1.09,42202.0,21.1786,5732294,242689,66752
3,488770,7,KODEX 머니마켓액티브,103195,2,5,0.00,103203.0,0.7714,316358,32645,64947
4,133690,4,TIGER 미국나스닥100,136295,2,305,0.22,136404.0,5.4480,173381,23608,51192
...,...,...,...,...,...,...,...,...,...,...,...,...
985,465620,4,ACE 미국빅테크TOP7 Plus인버스(합성),9660,2,50,0.52,9645.0,-11.5916,458,4,24
986,139310,5,TIGER 금속선물(H),5820,5,-55,-0.94,5880.0,-1.2606,3753,21,23
987,145670,3,ACE 인버스,4485,5,-50,-1.10,4485.0,-17.9186,10994,49,22
988,275750,3,RISE 코스닥150선물인버스,3665,5,-10,-0.27,3662.0,-10.5840,1988,7,22


In [10]:
# 복붙
def calculate_rsi(data, window = 14):
    delta = data.diff()
    gain = delta.where(delta > 0, 0).rolling(window = 14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window = 14).mean()
    RS = gain / (loss + 1e-10)
    RSI = 100 - (100 / (1 + RS))
    return RSI

In [20]:
from tqdm import tqdm

etfs_dic = {}
for i in tqdm(range(len(etfs))):

    try:
        row = etfs.iloc[i]
        symbol = row['Symbol']
        name = row['Name']
        df = fdr.DataReader(symbol)
        df['RSI'] = calculate_rsi(df['Close'])
        df = df.dropna()
        df = df[['Close', 'Change', 'RSI']]
        etfs_dic[symbol] = [name, df]
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 990/990 [02:07<00:00,  7.75it/s]


In [29]:
X = []
Y = []

window = 50

for symbol in tqdm(etfs_dic):
    name = etfs_dic[symbol][0]
    df = etfs_dic[symbol][1].copy().values
    for i in range(len(df) - window):
        A = df[i : i + window, 1:].flatten()      # Change, RSI열만 학습
        before = df[i + window - 1, 0]
        after = df[i + window, 0]
        B = (after - before) / before * 100 >= 5
        if len(A) == 100:
            X.append(A)
            Y.append(B)

100%|███████████████████████████████████████████████████████████████████████████████| 908/908 [00:02<00:00, 376.99it/s]


In [30]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier  # 데이터가 불균형해서 부스팅 모델 쓰기로 했음
from sklearn.metrics import classification_report

train_x, test_x, train_y, test_y = train_test_split(X, Y, stratify=Y)

model = XGBClassifier()
model.fit(train_x, train_y)

pred = model.predict(test_x)

report = classification_report(test_y, pred)
print(report)

# 성능 70% 이상으로 올라감

              precision    recall  f1-score   support

       False       1.00      1.00      1.00    266769
        True       0.78      0.38      0.52      1546

    accuracy                           1.00    268315
   macro avg       0.89      0.69      0.76    268315
weighted avg       1.00      1.00      1.00    268315



In [31]:
model.fit(X, Y)   # 재학습

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [37]:
for symbol in etfs_dic:
    try:
        name = etfs_dic[symbol][0]
        df = etfs_dic[symbol][1].copy()[["Change", "RSI"]].values[-window:].flatten()        # 최신데이터 50개만 갖고온 것
        pred = model.predict([df])
        if pred[0] == 1:
            print(name)
    except:
        continue
# 다음날 5% 이상 오르는거 찾음 (무조건 맞진 X. 78% 확률)

KODEX 2차전지산업레버리지
TIGER 200에너지화학레버리지
